In [1]:
# Import the required libraries.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re, random
from copy import copy, deepcopy
import sys
from collections import Counter
import queue
from scipy import stats
from scipy.special import comb
from sklearn.metrics import precision_recall_fscore_support
np.random.seed(21)
from sklearn.model_selection import train_test_split
random.seed(21)

In [2]:
fz = frozenset

In [3]:
def compute_entropy(labels):
    entropy = 0.0
    totSamples = len(labels)
    labelSet = set(labels.reshape(-1))
    for label in labelSet:
        prob = np.sum(labels == label) / totSamples
        if prob > 1e-12:
            entropy -= np.log(prob) * prob

    return entropy

In [4]:
def get_entropy(data_i, labels):
    attr_split_info = 0
    attr_count = dict()
    for attr_val in set(data_i.reshape(-1)):
        ids = np.where(data_i == attr_val)[0]
        attr_count[attr_val] = len(ids)
        attr_split_info += attr_count[attr_val] * compute_entropy(labels[ids])
    attr_split_info /= len(data_i)
    return attr_split_info

In [5]:
def getBestRuleCN2(all_cond,dataset,labels):
    # aim :to reduce entropy. ie choose the one with minimum entropy.
    # also gain must be positive. so the weighted entropy must be lesser than old entropy.
    # condition of frm (idx, val) and evaluated at dataset[:,idx] == val
    min_significant = 2
    max_rules_count = 10 # to use in beam search
    old_entopy = compute_entropy(labels)
    min_entropy = float('inf')
    best_rule_set = None
    best_rule_mask = None
    candidates = {} # candidates are of the form {ruleset : (entropy,mask)} 
    # initial ruleset empty, inital mask: all true initial entropy: old entropy
    # Done: give initial candidates
    emp_set = set()
    emp_set = fz(emp_set)
    all_true_mask = np.asarray([True]*dataset.shape[0],dtype=np.bool)
    candidates[emp_set] = (old_entopy, all_true_mask) 
    
    while len(candidates)!= 0:
        next_candidates = dict()
        for rule_fz, tup in candidates.items():
            rule_set = set(rule_fz)
            rule_mask = tup[1]
            rule_entropy = tup[0]
            rule_attr = set([cond[0] for cond in rule_set])
            for new_cond in all_cond:
                if new_cond[0] in rule_attr: # preventing collisions /inconsistencies {A_i = v1, A_i = v2}
                    continue
                new_rule_mask = rule_mask & dataset[:,new_cond[0]] == new_cond[1]
                if np.sum(new_rule_mask) > min_significant:  # checking significance
                    new_rule_entropy = get_entropy(new_rule_mask,labels)
                    new_rule = deepcopy(rule_set)
                    new_rule.add(new_cond)
                    new_rule = fz(new_rule)
                    next_candidates[new_rule] = (new_rule_entropy, new_rule_mask) # map takes care of dupelicates
                    if(new_rule_entropy < min_entropy): # is this right? can entropy decrease later? yes hence beam search. corrected.
                        min_entropy = new_rule_entropy
                        best_rule_set = new_rule
                        best_rule_mask = new_rule_mask
                        # add to next candidates, check for best
        att_tosort = {k:dum[0] for k,dum in next_candidates.items()}
        sort_next_candidates = [ k for k in sorted(att_tosort, key=att_tosort.get)]
        sort_next_candidates = sort_next_candidates[:max_rules_count]
        candidates= {}
        for k in sort_next_candidates:
            candidates[k] = next_candidates[k]
    if min_entropy < old_entopy:
        return best_rule_set, best_rule_mask
    else:
#         print(min_entropy, old_entopy)
        return None

In [6]:
def CN2_train(dataset,labels,all_conds):
    # ordered rules is followed
    rule_list = []
    dataset = deepcopy(dataset)
    labels = deepcopy(labels)
    dataset_majority_class = Counter(labels.reshape(-1)).most_common(1)[0][0]
    
    while(len(labels.reshape(-1)) != 0):
        next_rule_set = getBestRuleCN2(all_conds, dataset, labels)
        if next_rule_set is None:
            break
        
        to_delete = next_rule_set[1]
        to_keep = (to_delete == False)
        delete_labels = labels[to_delete]
        majority_class = Counter(delete_labels.reshape(-1)).most_common(1)[0][0]
        rule_list.append((next_rule_set[0],majority_class))
        dataset = dataset[to_keep]
        labels = labels[to_keep]
#         print(labels.shape)
    return rule_list, dataset_majority_class
        

In [7]:
def get_to_check_tuples(rule_list):
    to_check_tuples = []
    for rule,maj_class in rule_list:
        attr_list = np.asarray([d[0] for d in rule], dtype=np.int32)
        val_list = np.asarray([d[1] for d in rule])
        to_check_tuples.append( (attr_list, val_list, maj_class))
    return to_check_tuples
        
            

In [8]:
def rule_infer(dataset,rule_list, default_class = -1):
    labels = np.zeros((dataset.shape[0],), dtype = np.int32)
    labels += default_class;
    to_check_tuples = get_to_check_tuples(rule_list)
    for idx, sample in enumerate(dataset):
        for attr, val, maj_cls in to_check_tuples:
            if np.all(sample[attr] == val):
                labels[idx] = maj_cls
                break;
    return labels
  

Minimum description length derived from http://www.csee.usf.edu/~lohall/dm/ripper.pdf

In [9]:
def foil_info_gain(p_new, n_new, p_old, n_old):
    gain = np.log2(p_new / (p_new+n_new)) - np.log2(p_old / (p_old+n_old))
    gain = p_new*gain # acc to slides, multiply by t : no of positive by both old and new rule = p1 bcoz subset superset property
    return gain

In [10]:
def calculate_dl(rule_len, max_rules, p_count, n_count, fp_count, fn_count):
    l_model = 0
    if rule_len != 0:
        l_model = rule_len * np.log2(max_rules/ rule_len) + (max_rules - rule_len)* np.log2(max_rules / (max_rules- rule_len))
        l_model += np.log2(rule_len) # encode rule
        l_model /= 2 # redundancy removal
    
    n_comb = np.asarray([p_count, n_count])
    k_comb = np.asarray([fp_count, fn_count])
    l_error = np.sum(np.log2(comb(n_comb, k_comb)))
    
    return l_model + l_error

In [11]:
def calculate_prune_val(p, n):
    return (p-n)/(p+n)

In [12]:
def get_mask_ignore(ruleset, data, ignore_cond = (-1,-1)):
    pos_mask = np.asarray([True]*data.shape[0], dtype = np.bool)
    for cond in ruleset:
        if cond == ignore_cond:
            continue
        pos_mask = pos_mask & (data[:,cond[0]] == cond[1])
    return np.asarray(pos_mask)

In [13]:
def get_rule_Ripper(dataset, labels, pos_class, all_cond):
    pos_idx = np.where(labels == pos_class)[0]
    neg_idx = np.where(labels != pos_class)[0]
    all_pos, all_neg = dataset[pos_idx], dataset[neg_idx]
    
    grow_pos, prune_pos = train_test_split(all_pos, test_size=0.33, random_state=42)
    grow_neg, prune_neg = train_test_split(all_neg, test_size=0.33, random_state=42)
    
    rule_set = set()
    rule_attr = set()
    
    # growing step
    
    while grow_neg.shape[0] > 0: # do when thre are negative samples
        best_cond = None
        best_gain = -1* float('inf')
        for new_cond in all_cond:
            if new_cond[0] in rule_attr:
                continue
            p_new = np.sum(grow_pos[:,new_cond[0]] == new_cond[1])
            n_new = np.sum(grow_neg[:,new_cond[0]] == new_cond[1])
            
            new_gain = foil_info_gain(p_new, n_new, grow_pos.shape[0], grow_neg.shape[0])
            if new_gain > best_gain:
                best_gain = new_gain
                best_cond = new_cond
            
        if best_gain < 0:
            break
        rule_set.add(best_cond)
        rule_attr.add(best_cond[0])
        new_grow_pos_mask = grow_pos[:,best_cond[0]] == best_cond[1]
        new_grow_neg_mask = grow_neg[:,best_cond[0]] == best_cond[1]
        grow_pos = grow_pos[new_grow_pos_mask]
        grow_neg = grow_neg[new_grow_neg_mask]
        
    if len(rule_set) == 0:
        return None
    # print('rule_len_b4prune:  ',len(rule_set))
    # pruning step
    pos_rule_mask = get_mask_ignore(rule_set,prune_pos,(-1,-1))
    neg_rule_mask = get_mask_ignore(rule_set,prune_neg,(-1,-1))
    rule_v = calculate_prune_val(np.sum(pos_rule_mask), np.sum(neg_rule_mask))
    # print('rule_val', rule_v, np.sum(pos_rule_mask), np.sum(neg_rule_mask))
    while len(rule_set) != 0:
        worst_cond = None
        max_new_val = -1*float('inf')
        for cond in rule_set:
            new_pos_p = np.sum(get_mask_ignore(rule_set,prune_pos,cond))
            new_neg_n = np.sum(get_mask_ignore(rule_set,prune_neg,cond))
            new_rule_v = calculate_prune_val(new_pos_p, new_neg_n)
            if new_rule_v > max_new_val:
                max_new_val = new_rule_v
                worst_cond = cond
        if max_new_val >= rule_v:
            rule_v = max_new_val
            rule_set.remove(worst_cond)
            rule_attr.remove(worst_cond[0])
            # print('removing', worst_cond)
        else: # ie reduce till you can increase the value v.
            # print(max_new_val, worst_cond, rule_v)
            break
    
    if len(rule_set) == 0:
        return None
    # todo error rate:
    pos_mask = get_mask_ignore(rule_set,prune_pos,(-1,-1))
    neg_mask = get_mask_ignore(rule_set,prune_neg,(-1,-1))
    neg_mask = (neg_mask==False)
    acc = np.sum(pos_mask) 
    acc += np.sum(neg_mask)
    acc /= (len(pos_mask) + len(neg_mask))
    error_rate = 1-acc
    if error_rate > 0.5:
        # print(error_rate)
        return None
    
    return rule_set, get_mask_ignore(rule_set, dataset, (-1,-1)) 
                    

In [14]:
def coverage(nc,n):
    return nc/n

def accuracy(ncorr,n):
    return ncorr/n

def laplace(nc,n,k=2):
    return (nc+1)/(n+k)

def m_estimate(nc, n,p = 0.5,k =2):
    return (nc+ (k*p))/(n+k)

def all_metrics(nc,n,ncorr,k=2):
    return [accuracy(nc,n), coverage(ncorr,n), laplace(nc,n,k)]

In [15]:
def ripper_train(dataset,labels,all_conds):
    # ordered rule is followed for each class
    # move on to the next class only after the current minority class is exhausted.
    #default class is the majority class
    
    rule_list = []
    dataset = deepcopy(dataset)
    labels = deepcopy(labels)
    class_freq = Counter(labels.reshape(-1)).most_common()
    majority_class = class_freq[0]
    class_freq = class_freq[1:]
    class_freq.reverse()
    
    for pos_class, _ in class_freq:
        while np.any(labels.reshape(-1) == pos_class):
            if len(dataset)==0:
                break
            next_rule = get_rule_Ripper(dataset, labels, pos_class, all_cond)
#             print(next_rule)
            if next_rule is None:
                break
            to_delete = next_rule[1]
            # print(np.sum(to_delete))
            to_keep = (to_delete == False)
            delete_labels = labels[to_delete]
            rule_list.append((next_rule[0], pos_class))
            # print(rule_list[-1])
            dataset = dataset[to_keep]
            labels = labels[to_keep]
            
    return rule_list, majority_class[0]
        

In [16]:
train_df = pd.read_csv('train.csv')

print(train_df.columns.values)

['PassengerId' 'Survived' 'Pclass' 'Name' 'Sex' 'Age' 'SibSp' 'Parch'
 'Ticket' 'Fare' 'Cabin' 'Embarked']


In [17]:
train_df['Has_Cabin'] = train_df["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
train_df['Embarked'] = train_df['Embarked'].fillna('S')
# train_df = train_df.drop(['PassengerId','Name', 'Age', 'Ticket', 'Fare', 'Cabin'], axis=1)
train_df['Fare'] = train_df['Fare'].fillna(train_df['Fare'].median())
train_df['CategoricalFare'] = pd.qcut(train_df['Fare'], 4)

In [18]:
age_avg = train_df['Age'].mean()
age_std = train_df['Age'].std()
age_null_count = train_df['Age'].isnull().sum()
age_null_random_list = np.random.randint(age_avg - age_std, age_avg + age_std, size=age_null_count)
train_df['Age'][np.isnan(train_df['Age'])] = age_null_random_list
train_df['Age'] = train_df['Age'].astype(int)
train_df['CategoricalAge'] = pd.cut(train_df['Age'], 5)

/home/shikhar/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [19]:
for dataset in [train_df]:
    # Mapping Sex
    dataset['Sex'] = dataset['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
    
    # Mapping Embarked
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
    
    # Mapping Fare
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare'] = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)
    
    # Mapping Age
    dataset.loc[ dataset['Age'] <= 16, 'Age'] 					       = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age'] = 4 ;

In [20]:
drop_elements = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp']
train_df = train_df.drop(drop_elements, axis = 1)
train_df = train_df.drop(['CategoricalAge', 'CategoricalFare'], axis = 1)

In [21]:
print(train_df.columns.values)

['Survived' 'Pclass' 'Sex' 'Age' 'Parch' 'Fare' 'Embarked' 'Has_Cabin']


In [22]:
id2attr = {i: attribute for i, attribute in enumerate(train_df.columns.values[1:])}

condition2string = {(0, 1): 'Pclass is 1', (0, 2): 'Pclass is 2', (0, 3): 'Pclass is 3',
                    (1, 0): 'Sex is Female', (1, 1): 'Sex is Male',
                    (2, 0): 'Age <= 16', (2, 1): '16 < Age <= 32', (2, 2): '32 < Age <= 48', (2, 3): '48 < Age <= 64', (2, 4): 'Age > 64',
                    (3, 0): '#Parent/Child is 0', (3, 1): '#Parent/Child is 1', (3, 2): '#Parent/Child is 2', (3, 3): '#Parent/Child is 3', (3, 4): '#Parent/Child is 4', (3, 5): '#Parent/Child is 5', (3, 6): '#Parent/Child is 6',
                    (4, 0): 'Fare <= 7.91', (4, 1): '7.91 < Fare <= 14.454', (4, 2): '14.454 < Fare <= 31.0', (4, 3): 'Fare > 31.0',
                    (5, 0): 'Embarked from Southampton', (5, 1): 'Embarked from Cherbourg', (5, 2): 'Embarked from Queenstown',
                    (6, 1): 'Has a cabin', (6, 0): 'Doesn\'t have a cabin'}

In [23]:
tfn = train_df.values

In [24]:
x,y = tfn[:,1:], tfn[:,0]

In [25]:
poss_val_dict = dict()
for i in range(x.shape[1]):
    poss_val = list(set(x[:,i]))
    poss_val_dict[i] = poss_val

In [26]:
all_cond = []
for lhs, all_rhs in poss_val_dict.items():
    dum = [(lhs, b) for b in all_rhs]
    all_cond = all_cond + dum

print(all_cond)        

[(0, 1), (0, 2), (0, 3), (1, 0), (1, 1), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (4, 0), (4, 1), (4, 2), (4, 3), (5, 0), (5, 1), (5, 2), (6, 0), (6, 1)]


In [27]:
x_tr, x_te,y_tr,y_te = train_test_split(x,y, test_size=0.2, random_state= 96)

In [28]:
x_tr.shape , y_tr.shape

((712, 7), (712,))

In [29]:
out = CN2_train(x_tr,y_tr,deepcopy(all_cond))

In [30]:
len(out[0])

3

In [31]:
y_hat = rule_infer(x_te,out[0], default_class = out[1])

In [32]:
print('Accuracy of CN2: %0.4f'%(np.sum(y_hat == y_te) / len(y_hat)))

Accuracy of CN2: 0.7933


In [33]:
prior_p = {}
prior_p[0] = (np.sum(y_tr == 0) / len(y_tr))
prior_p[1] = 1 - prior_p[0]
k = 2
ruleset_metrics = [] # (no_of_rules, 6)
for rule in out[0]:
    rule_metrics = [] # (6,) => (3,) train (3,) test
    for data in [(x_tr,y_tr), (x_te,y_te)]:
        rule_mask = get_mask_ignore(rule[0], data[0])
        nc = np.sum(rule_mask)
        n = len(rule_mask)
        y_hat = rule_infer(data[0],[rule], default_class = out[1])
        ncorr = np.sum(y_hat == data[1])
        rule_metrics += all_metrics(nc,n,ncorr,k)
    rule_metrics = np.asarray(rule_metrics, dtype=np.float32)
    ruleset_metrics.append(rule_metrics)
ruleset_metrics = np.asarray(ruleset_metrics, dtype=np.float32)

In [34]:
id2label = {0: 'Didn\'t Survive',
            1: 'Survived'}

for ruleno, rule in enumerate(out[0]):
    to_print = []
    
    for cond in rule[0]:
        to_print.append('( ' + condition2string[cond] + ' )')
    to_print = ["IF ", " AND ".join(to_print)]
    to_print.append(" THEN {}".format(id2label[rule[1]]))
    print(''.join(to_print))
    print('Accuracy:%.3f Coverage: %.3f, laplace: %.3f\n'%(ruleset_metrics[ruleno][3], ruleset_metrics[ruleno][4], ruleset_metrics[ruleno][5]))
print('ELSE: {}'.format(id2label[out[1]]))

IF ( Sex is Female ) THEN Survived
Accuracy:0.296 Coverage: 0.793, laplace: 0.298

IF ( Doesn't have a cabin ) THEN Didn't Survive
Accuracy:0.771 Coverage: 0.687, laplace: 0.768

IF ( Pclass is 1 ) AND ( #Parent/Child is 0 ) AND ( Sex is Female ) THEN Didn't Survive
Accuracy:0.045 Coverage: 0.687, laplace: 0.050

ELSE: Didn't Survive


In [35]:
out = ripper_train(x_tr,y_tr,deepcopy(all_cond))

/home/shikhar/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log2
  
/home/shikhar/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in multiply
  This is separate from the ipykernel package so we can avoid doing imports until
/home/shikhar/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in long_scalars
  


In [36]:
len(out[0])

5

In [37]:
y_hat = rule_infer(x_te,out[0], default_class = out[1])

In [38]:
print('Accuracy of Ripper: %0.4f'%(np.sum(y_hat == y_te) / len(y_hat)))

Accuracy of Ripper: 0.8156


In [39]:
prior_p = {}
prior_p[0] = (np.sum(y_tr == 0) / len(y_tr))
prior_p[1] = 1 - prior_p[0]
k = 2
ruleset_metrics = [] # (no_of_rules, 6)
for rule in out[0]:
    rule_metrics = [] # (6,) => (3,) train (3,) test
    for data in [(x_tr,y_tr), (x_te,y_te)]:
        rule_mask = get_mask_ignore(rule[0], data[0])
        nc = np.sum(rule_mask)
        n = len(rule_mask)
        y_hat = rule_infer(data[0],[rule], default_class = out[1])
        ncorr = np.sum(y_hat == data[1])
        rule_metrics += all_metrics(nc,n,ncorr,k)
    rule_metrics = np.asarray(rule_metrics, dtype=np.float32)
    ruleset_metrics.append(rule_metrics)
ruleset_metrics = np.asarray(ruleset_metrics, dtype=np.float32)
        
        

In [40]:
id2label = {0: 'Didn\'t Survive',
            1: 'Survived'}

for ruleno, rule in enumerate(out[0]):
    to_print = []
    
    for cond in rule[0]:
        to_print.append('( ' + condition2string[cond] + ' )')
    to_print = ["IF ", " AND ".join(to_print)]
    to_print.append(" THEN {}".format(id2label[rule[1]]))
    print(''.join(to_print))
    print('Accuracy:%.3f Coverage: %.3f, laplace: %.3f\n'%(ruleset_metrics[ruleno][3], ruleset_metrics[ruleno][4], ruleset_metrics[ruleno][5]))
print('ELSE: {}'.format(id2label[out[1]]))

IF ( Pclass is 1 ) AND ( Sex is Female ) THEN Survived
Accuracy:0.067 Coverage: 0.732, laplace: 0.072

IF ( Sex is Female ) AND ( Pclass is 2 ) THEN Survived
Accuracy:0.073 Coverage: 0.737, laplace: 0.077

IF ( 14.454 < Fare <= 31.0 ) AND ( Sex is Female ) AND ( Embarked from Queenstown ) THEN Survived
Accuracy:0.006 Coverage: 0.682, laplace: 0.011

IF ( Sex is Female ) AND ( Fare <= 7.91 ) THEN Survived
Accuracy:0.050 Coverage: 0.715, laplace: 0.055

IF ( Age <= 16 ) AND ( Has a cabin ) THEN Survived
Accuracy:0.022 Coverage: 0.698, laplace: 0.028

ELSE: Didn't Survive
